In [1]:
import os
import xml.etree.ElementTree as ET
# XML 파일들이 있는 경로
input_dir = 'C:/Users/AI-LHJ/Desktop/EGG/Training/label'  # 사용자에 맞게 경로를 변경해주세요
output_dir = 'C:/Users/AI-LHJ/Desktop/EGG/Training/labels'  # YOLO 형식의 라벨을 저장할 폴더
os.makedirs(output_dir, exist_ok=True)
files = [f for f in os.listdir(input_dir) if f.endswith('.xml')]
for file in files:
    # XML 파일 파싱
    tree = ET.parse(os.path.join(input_dir, file))
    root = tree.getroot()

    # 이미지 크기 정보
    img_width = int(root.find('size/width').text)
    img_height = int(root.find('size/height').text)

    # YOLO 형식 데이터로 변환
    yolo_data = []
    for obj in root.findall('bndbox'):
        # 클래스 ID 가져오기 (여기서는 'state' 태그를 클래스 ID로 사용)
        class_id = int(obj.find('state').text) - 1  # YOLO는 0부터 시작하므로 1을 뺌

        # 바운딩 박스 정보
        x_min = float(obj.find('x_min').text)
        y_min = float(obj.find('y_min').text)
        x_max = float(obj.find('x_max').text)
        y_max = float(obj.find('y_max').text)

        # 중심점, 너비, 높이 계산
        x_center = (x_min + x_max) / 2.0 / img_width
        y_center = (y_min + y_max) / 2.0 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        # YOLO 형식의 라벨 데이터 추가
        yolo_data.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # 원본 파일명에서 확장자를 .txt로 변경하여 저장
    output_file = os.path.join(output_dir, os.path.splitext(file)[0] + '.txt')
    with open(output_file, 'w') as f:
        f.write("\n".join(yolo_data))

print(f"YOLO 형식의 라벨 파일들이 {output_dir}에 저장되었습니다.")

YOLO 형식의 라벨 파일들이 C:/Users/AI-LHJ/Desktop/EGG/Training/labels에 저장되었습니다.


In [6]:
import os
import random
import shutil

# 경로 설정
base_path = r"C:/Users/AI-LHJ/Desktop/EGG/Training"
train_path = os.path.join(base_path, 'train')
valid_path = os.path.join(base_path, 'vaild')
test_path = os.path.join(base_path, 'test')

# 이미지와 라벨 파일 경로
train_images_path = os.path.join(train_path, 'images')
train_labels_path = os.path.join(train_path, 'labels')

valid_images_path = os.path.join(valid_path, 'images')
valid_labels_path = os.path.join(valid_path, 'labels')

test_images_path = os.path.join(test_path, 'images')
test_labels_path = os.path.join(test_path, 'labels')

# 폴더가 없으면 생성
os.makedirs(valid_images_path, exist_ok=True)
os.makedirs(valid_labels_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)

# train 폴더에 있는 모든 이미지 파일 리스트
images = os.listdir(train_images_path)

# 파일을 랜덤하게 섞기
random.shuffle(images)

# 데이터 비율 계산 (7:2:1)
total = len(images)
train_split = int(total * 0.7)
valid_split = int(total * 0.9)  # 70% train + 20% valid

# train:valid:test로 나누기
train_images = images[:train_split]
valid_images = images[train_split:valid_split]
test_images = images[valid_split:]

def move_files(file_list, src_images, src_labels, dst_images, dst_labels):
    for image_file in file_list:
        # 이미지 파일 이동
        src_image = os.path.join(src_images, image_file)
        dst_image = os.path.join(dst_images, image_file)
        shutil.move(src_image, dst_image)

        # 라벨 파일은 이미지 파일과 동일한 이름에 확장자만 .txt
        label_file = image_file.replace('.jpg', '.txt')  # 확장자에 맞게 수정 (.jpg -> .txt)
        src_label = os.path.join(src_labels, label_file)
        dst_label = os.path.join(dst_labels, label_file)

        # 라벨 파일이 존재할 경우 이동
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
        else:
            print(f"Warning: {label_file} 파일을 찾을 수 없습니다.")

# 파일 이동 (train -> valid, test)
move_files(valid_images, train_images_path, train_labels_path, valid_images_path, valid_labels_path)
move_files(test_images, train_images_path, train_labels_path, test_images_path, test_labels_path)

print("데이터가 7:2:1 비율로 나누어졌습니다.")


데이터가 7:2:1 비율로 나누어졌습니다.


In [1]:
from ultralytics import YOLO
from multiprocessing import freeze_support

# 모델 경로 설정
model_path = 'yolov8m.pt'
# 데이터 설정 파일 경로
data_path = 'C:/Users/AI-LHJ/Desktop/EGG/data.yaml'
# 학습 결과 저장 경로
project_path = 'C:/Users/AI-LHJ/Desktop/EGG/train_result'

# YOLO 모델 불러오기
model = YOLO(model_path)

if __name__ == '__main__':
    freeze_support()
    
    # 모델 학습
    model.train(data=data_path, epochs=100, project=project_path, name='color_egg_m')

New https://pypi.org/project/ultralytics/8.2.100 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.8.16 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:/Users/AI-LHJ/Desktop/EGG/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=C:/Users/AI-LHJ/Desktop/EGG/train_result, name=color_egg_m2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms

train: Scanning C:\Users\AI-LHJ\Desktop\EGG\train\labels.cache... 30202 images, 36 backgrounds, 0 corrupt: 100%|██████████| 30202/30202 [00:00<?, ?it/s]
val: Scanning C:\Users\AI-LHJ\Desktop\EGG\vaild\labels.cache... 8629 images, 13 backgrounds, 0 corrupt: 100%|██████████| 8629/8629 [00:00<?, ?it/s]


Plotting labels to C:\Users\AI-LHJ\Desktop\EGG\train_result\color_egg_m2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\AI-LHJ\Desktop\EGG\train_result\color_egg_m2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100       7.1G      0.261     0.6622     0.8363        159        640: 100%|██████████| 1888/1888 [06:07<00:00,  5.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 270/270 [00:44<00:00,  6.06it/s]

                   all       8629      82556      0.806       0.83      0.881      0.879


...

     65/100       6.8G     0.1734     0.2828     0.8035        146        640: 100%|██████████| 1888/1888 [06:04<00:00,  5.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 270/270 [00:46<00:00,  5.84it/s]

                   all       8629      82556      0.904      0.889      0.954      0.953



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     66/100      6.79G     0.1736     0.2815      0.804        275        640:  37%|███▋      | 700/1888 [02:18<03:54,  5.07it/s]


KeyboardInterrupt: 

In [2]:
from ultralytics import YOLO

# 모델 로드
model = YOLO("C:/Users/AI-LHJ/Desktop/ultralytics-main/runs/detect/train/weights/best.pt")  # 사용자 정의 모델을 로드합니다

# 모델 검증
metrics = model.val()  # 인자가 필요 없음, 데이터셋과 설정이 기억됩니다
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # 각 카테고리의 map50-95가 포함된 목록

Ultralytics YOLOv8.1.34 🚀 Python-3.8.16 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 268 layers, 43608921 parameters, 0 gradients, 164.8 GFLOPs


val: Scanning C:\Users\AI-LHJ\Desktop\Wire\valid\labels.cache... 195 images, 0 backgrounds, 0 corrupt: 100%|██████████| 195/195 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:03<00:00,  3.95it/s]


                   all        195       2366      0.864      0.807      0.839      0.647
              crossarm        195        422      0.944       0.86      0.936      0.764
                  polo        195        196      0.943       0.98      0.982      0.724
                  wire        195       1748      0.706      0.581        0.6      0.452
Speed: 1.0ms preprocess, 10.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val2


array([    0.76412,     0.72417,     0.45224])